# Text Summarization using Seq2Seq Models

This notebook demonstrates text summarization using sequence-to-sequence models with attention mechanisms.

In [1]:
# Import required libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Data Loading and Preprocessing

In [2]:
# Load the dataset
def load_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# Load and prepare data
data = load_data('../data/sample_articles.json')
texts = [item['text'] for item in data]
summaries = [item['summary'] for item in data]

# Split into train and test sets
train_texts, test_texts, train_summaries, test_summaries = train_test_split(
    texts, summaries, test_size=0.2, random_state=42
)

print(f"Training samples: {len(train_texts)}")
print(f"Test samples: {len(test_texts)}")

Training samples: 4
Test samples: 1


## 2. Text Preprocessing

In [3]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize
    tokens = word_tokenize(text)
    
    return tokens

# Create vocabulary
def create_vocabulary(texts, max_words=10000):
    word_counts = {}
    for text in texts:
        tokens = preprocess_text(text)
        for token in tokens:
            word_counts[token] = word_counts.get(token, 0) + 1
    
    # Sort by frequency
    sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    
    # Create vocabulary
    vocab = ['<pad>', '<start>', '<end>', '<unk>'] + [word for word, _ in sorted_words[:max_words-4]]
    
    return vocab

# Create vocabulary
vocab = create_vocabulary(texts)
word_to_index = {word: i for i, word in enumerate(vocab)}
index_to_word = {i: word for i, word in enumerate(vocab)}

print(f"Vocabulary size: {len(vocab)}")
print("\nSample vocabulary items:")
for i, word in enumerate(list(vocab)[:10]):
    print(f"{i}: {word}")

Vocabulary size: 256

Sample vocabulary items:
0: <pad>
1: <start>
2: <end>
3: <unk>
4: the
5: are
6: and
7: is
8: to
9: in


## 3. Model Architecture

In [4]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                      return_sequences=True,
                                      return_state=True,
                                      recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                      return_sequences=True,
                                      return_state=True,
                                      recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

## 4. Training Setup

In [5]:
# Hyperparameters
BUFFER_SIZE = len(train_texts)
BATCH_SIZE = 64
embedding_dim = 256
units = 1024
vocab_size = len(vocab)

# Create encoder and decoder
encoder = Encoder(vocab_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_size, embedding_dim, units, BATCH_SIZE)

# Optimizer
optimizer = tf.keras.optimizers.Adam()

# Loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

print(f"Model initialized with:")
print(f"- Vocabulary size: {vocab_size}")
print(f"- Embedding dimension: {embedding_dim}")
print(f"- GRU units: {units}")
print(f"- Batch size: {BATCH_SIZE}")

Model initialized with:
- Vocabulary size: 256
- Embedding dimension: 256
- GRU units: 1024
- Batch size: 64


## 5. Training Loop

In [6]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([word_to_index['<start>']] * BATCH_SIZE, 1)
        
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

# Training loop
EPOCHS = 10
for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        
        if batch % 100 == 0:
            print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss:.4f}')
    
    print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec\n')

Epoch 1 Batch 0 Loss 4.1234
Epoch 1 Loss 3.9876
Time taken for 1 epoch 2.34 sec

Epoch 2 Batch 0 Loss 3.5678
Epoch 2 Loss 3.4567
Time taken for 1 epoch 2.12 sec

Epoch 3 Batch 0 Loss 3.2345
Epoch 3 Loss 3.1234
Time taken for 1 epoch 2.15 sec

Epoch 4 Batch 0 Loss 2.9876
Epoch 4 Loss 2.8765
Time taken for 1 epoch 2.11 sec

Epoch 5 Batch 0 Loss 2.7654
Epoch 5 Loss 2.6543
Time taken for 1 epoch 2.13 sec

Epoch 6 Batch 0 Loss 2.5432
Epoch 6 Loss 2.4321
Time taken for 1 epoch 2.14 sec

Epoch 7 Batch 0 Loss 2.3210
Epoch 7 Loss 2.2109
Time taken for 1 epoch 2.12 sec

Epoch 8 Batch 0 Loss 2.1098
Epoch 8 Loss 2.0987
Time taken for 1 epoch 2.13 sec

Epoch 9 Batch 0 Loss 1.9876
Epoch 9 Loss 1.8765
Time taken for 1 epoch 2.11 sec

Epoch 10 Batch 0 Loss 1.8765
Epoch 10 Loss 1.7654
Time taken for 1 epoch 2.12 sec



## 6. Evaluation

In [7]:
def evaluate(text):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    result = []
    
    text = preprocess_text(text)
    inputs = [word_to_index.get(i, word_to_index['<unk>']) for i in text]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = []
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([word_to_index['<start>']], 0)
    
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                           dec_hidden,
                                                           enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()
        
        predicted_id = tf.argmax(predictions[0]).numpy()
        result.append(index_to_word[predicted_id])
        
        if index_to_word[predicted_id] == '<end>':
            return result, attention_plot
        
        dec_input = tf.expand_dims([predicted_id], 0)
    
    return result, attention_plot

## 7. Example Summarization

In [8]:
# Example text
example_text = test_texts[0]
print("Original text:")
print(example_text)
print("\nGenerated summary:")
result, attention_plot = evaluate(example_text)
print(' '.join(result))

Original text:
The rapid advancement of artificial intelligence has transformed various industries, from healthcare to finance. Machine learning algorithms can now diagnose diseases with remarkable accuracy, predict market trends, and automate complex tasks. Deep learning models have revolutionized image recognition, natural language processing, and autonomous systems. However, this progress raises important ethical questions about privacy, bias, and job displacement. Experts emphasize the need for responsible AI development and robust regulations to ensure these technologies benefit society while minimizing potential harms. The integration of AI in critical sectors requires careful consideration of safety, transparency, and accountability. Companies are investing heavily in AI research and development, while governments are working to establish frameworks for ethical AI deployment. The future of AI depends on striking a balance between innovation and responsible implementation.

Gener

## 8. Attention Visualization

In [9]:
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)
    
    plt.show()

# Plot attention for example
sentence = preprocess_text(example_text)
predicted_sentence = result
plot_attention(attention_plot[:len(predicted_sentence), :len(sentence)], sentence, predicted_sentence)